# ***Libraries***

In [1]:
pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 74.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import gc
import joblib
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.preprocessing import RobustScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer

import torch
import scipy.sparse
from scipy.sparse import load_npz
from scipy.sparse import diags
from sklearn.decomposition import TruncatedSVD, PCA
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

2025-05-15 20:10:03.384006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747339803.575935      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747339803.633321      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# ***Loading Data***

In [3]:
data_path = "/kaggle/input/jobs-descriptions-csv/job_descriptions.csv"
df = pd.read_csv(data_path)

df.head(2)

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Job Id            1615940 non-null  int64  
 1   Experience        1615940 non-null  object 
 2   Qualifications    1615940 non-null  object 
 3   Salary Range      1615940 non-null  object 
 4   location          1615940 non-null  object 
 5   Country           1615940 non-null  object 
 6   latitude          1615940 non-null  float64
 7   longitude         1615940 non-null  float64
 8   Work Type         1615940 non-null  object 
 9   Company Size      1615940 non-null  int64  
 10  Job Posting Date  1615940 non-null  object 
 11  Preference        1615940 non-null  object 
 12  Contact Person    1615940 non-null  object 
 13  Contact           1615940 non-null  object 
 14  Job Title         1615940 non-null  object 
 15  Role              1615940 non-null  object 
 16  

# ***Data Preprocessing***

## **1st | Dropping useless columns**

In [5]:
cols_to_drop = ['Job Id', 'Contact', 'Contact Person', 'Company Profile', 'Job Posting Date']

df.drop(columns=cols_to_drop, inplace=True)

## **2nd | Feature Engineering**

### **Part 1 | Discrete Categorical Columns**

#### 1) Column: ***Experience***

In [6]:
def extract_experience(exp_str):
    match = re.findall(r'\d+', str(exp_str))
    if len(match) == 2:
        return int(match[0]), int(match[1])
    elif len(match) == 1:
        return int(match[0]), int(match[0])
    else:
        return 0, 0

df[['exp_min', 'exp_max']] = df['Experience'].apply(lambda x: pd.Series(extract_experience(x)))
df['exp_avg'] = df[['exp_min', 'exp_max']].mean(axis=1)

df.drop('Experience', axis=1, inplace=True)
df.head(1)

,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,Preference,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,exp_min,exp_max,exp_avg
0,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,Female,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,5,15,10.0


#### 2) Column: ***Salary Range***

In [7]:
def extract_salary(sal_str):
    try:
        nums = re.findall(r'\d+', str(sal_str).replace('K', '000'))
        if len(nums) == 2:
            return int(nums[0]), int(nums[1])
        elif len(nums) == 1:
            return int(nums[0]), int(nums[0])
        else:
            return np.nan, np.nan
    except:
        return np.nan, np.nan

df[['salary_min', 'salary_max']] = df['Salary Range'].apply(lambda x: pd.Series(extract_salary(x)))
df['salary_avg'] = df[['salary_min', 'salary_max']].mean(axis=1)

df.drop("Salary Range", axis=1, inplace=True)
df.head(1)

,Qualifications,location,Country,latitude,longitude,Work Type,Company Size,Preference,Job Title,Role,...,Benefits,skills,Responsibilities,Company,exp_min,exp_max,exp_avg,salary_min,salary_max,salary_avg
0,M.Tech,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,Female,Digital Marketing Specialist,Social Media Manager,...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,5,15,10.0,59000,99000,79000.0


#### 3) Columns: ***Latitude & Longitude***

In [8]:
grid_size = 10
lat_col, lon_col = "latitude", "longitude"

lat_bins = int(180 // grid_size)  # 90 to -90
lon_bins = int(360 // grid_size)  # -180 to 180

# 0 at +90, increasing southward -- vertical
df['lat_index'] = ((90 - df[lat_col]) // grid_size).astype(int)

# 0 at -180, increasing eastward -- horizontal
df['lon_index'] = ((df[lon_col] + 180) // grid_size).astype(int)

# Region ID from top-left to bottom-right
df['geo_region_id'] = df['lat_index'] * lon_bins + df['lon_index']


df.drop(['lat_index','lon_index','latitude','longitude'], axis=1, inplace=True)
df.head(1)

,Qualifications,location,Country,Work Type,Company Size,Preference,Job Title,Role,Job Portal,Job Description,...,skills,Responsibilities,Company,exp_min,exp_max,exp_avg,salary_min,salary_max,salary_avg,geo_region_id
0,M.Tech,Douglas,Isle of Man,Intern,26801,Female,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,...,"Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,5,15,10.0,59000,99000,79000.0,125


#### 4) Column: ***Location & Country***

In [9]:
df.drop(columns=['location'], inplace=True) # no need for 'location' anymore


# Column: Country (Label Encoding)
le_country = LabelEncoder()
df['Country'] = le_country.fit_transform(df['Country'])

df.head(1)

,Qualifications,Country,Work Type,Company Size,Preference,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,exp_min,exp_max,exp_avg,salary_min,salary_max,salary_avg,geo_region_id
0,M.Tech,92,Intern,26801,Female,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,5,15,10.0,59000,99000,79000.0,125


#### 5) Columns: ***Qualifications, Work Type, Preference (Gender), Job Portal and Company***

In [10]:
# Label Encoding
LE_cols = ['Qualifications', 'Work Type', 'Preference', 'Job Portal']
for col in LE_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Frequency Encoding
df['Company_freq'] = df['Company'].map(df['Company'].value_counts())
df.drop(columns=['Company'], inplace=True)

df.head(1)

,Qualifications,Country,Work Type,Company Size,Preference,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,exp_min,exp_max,exp_avg,salary_min,salary_max,salary_avg,geo_region_id,Company_freq
0,6,92,2,26801,1,Digital Marketing Specialist,Social Media Manager,11,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",5,15,10.0,59000,99000,79000.0,125,1911


#### 6) Columns: ***Job Title (Label Column) & Role***

In [11]:
le_job_title = LabelEncoder()
df['Job Title_encoded'] = le_job_title.fit_transform(df['Job Title'])

# Store mapping for reverse lookup
job_title_classes = dict(zip(le_job_title.classes_, le_job_title.transform(le_job_title.classes_)))

# LabelEncoder object
with open('/kaggle/working/label_encoder_job_title.pkl', 'wb') as file:
    pickle.dump(le_job_title, file)

# Mapping dictionary
with open('/kaggle/working/job_title_classes.pkl', 'wb') as file:
    pickle.dump(job_title_classes, file)

In [12]:
# Role is broader than Job Title, and the latter is already chosen as a label so we will frequency-encode this one
df['Role_freq'] = df['Role'].map(df['Role'].value_counts())
df.drop(columns=['Role'], inplace=True)

df.head(1)

,Qualifications,Country,Work Type,Company Size,Preference,Job Title,Job Portal,Job Description,Benefits,skills,...,exp_min,exp_max,exp_avg,salary_min,salary_max,salary_avg,geo_region_id,Company_freq,Job Title_encoded,Role_freq
0,6,92,2,26801,1,Digital Marketing Specialist,11,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...",...,5,15,10.0,59000,99000,79000.0,125,1911,31,13945


### **Part 2 | Encoding & Normalizing Rich Text Columns Using NLP Techniques**

#### 1) Columns: ***Job Description & Responsibilities***  

##### **Data Leakage Avoidance (1)**

In [13]:
# To avoid label leakage; the textual columns might contain the job title -> SBERT will trivially map them together
def replace_title_tokens(text, title, special_token="[JOB_TITLE]"):
    """
    Masking job titles in text by:
    1. Handling hyphenated/slashed titles as in "UX/UI Designer"
    2. Matching plurals and case variations
    3. Preventing partial word matches like in "engineer" vs "engineering"
    """
    if not isinstance(text, str) or not isinstance(title, str):
        return text

    # Preprocessing any title variants: hyphens/slashes/plurals
    title_variants = set()
    for variant in re.split(r'[/-]', title.lower()):  # Splitting hyphenated/slashed titles
        title_variants.add(variant.strip())
        title_variants.add(variant.strip() + 's')  # Simple pluralization

    # Generate -regex- patterns for all variants
    patterns = []
    for variant in title_variants:
        escaped = re.escape(variant)
        patterns.extend([
            fr'\b{escaped}\b',       # Exact match
            fr'\b{escaped}\w*\b',    # Match plurals
        ])

    # Substitute (replace) all matches (case-insensitive)
    text_lower = text.lower()
    for pattern in set(patterns):  # Deduplicate patterns
        text_lower = re.sub(pattern, special_token, text_lower)

    # Cleanup, enforce exact word matches
    title_words = set()
    for variant in title_variants:
        title_words.update(variant.split())  # Splitting multi-word titles

    words = text_lower.split()
    cleaned_words = [
        special_token if word in title_words else word
        for word in words
    ]
    return ' '.join(cleaned_words)

# Apply to all textual columns
text_columns = ['Job Description', 'Responsibilities', 'skills']
special_token = "[JOB_TITLE]"

for col in tqdm(text_columns, desc="Processing text columns"):
    df[f"cleaned_{col}"] = df.apply(
        lambda x: replace_title_tokens(x[col], x['Job Title'], special_token),
        axis=1
    )

# Verify replacement
sample = df.sample(3)
for _, row in sample.iterrows():
    print(f"\nOriginal({row['Job Title']}):\n{row['Job Description'][:200]}...")
    print(f"\nCleaned:\n{row['cleaned_Job Description'][:200]}...")

df.drop(columns=['Job Title'], inplace=True)

Processing text columns: 100%|██████████| 3/3 [04:14<00:00, 84.82s/it]



Original(Software Tester):
A Quality Assurance Analyst tests software and products to ensure they meet quality standards. They identify defects, report issues, and work with development teams to resolve problems....

Cleaned:
a quality assurance analyst tests [JOB_TITLE] and products to ensure they meet quality standards. they identify defects, report issues, and work with development teams to resolve problems....

Original(Aerospace Engineer):
Systems Engineers design and manage complex systems and infrastructure. They integrate various components and technologies to ensure optimal system performance and reliability....

Cleaned:
systems [JOB_TITLE] design and manage complex systems and infrastructure. they integrate various components and technologies to ensure optimal system performance and reliability....

Original(Purchasing Agent):
A Supply Chain Coordinator manages supply chain operations, coordinates logistics, and ensures the seamless flow of goods and materials throughout th

In [14]:
# BEFORE any SVD/TF-IDF embedding steps to avoid test data leakage
from sklearn.model_selection import train_test_split

# First split: 70% train, 00% temp
train_df, temp_df = train_test_split(
    df, 
    test_size=0.3, 
    stratify=df['Job Title_encoded'],
    random_state=42
)

# Second split: 15% dev, 15% test (which is 50% of temp for each)
dev_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['Job Title_encoded'],
    random_state=42
)

# Saving for later use to disk
train_df.to_csv('/kaggle/working/train_data.csv', index=False)
dev_df.to_csv('/kaggle/working/dev_data.csv', index=False)
test_df.to_csv('/kaggle/working/test_data.csv', index=False)

print(f"""
Train: {len(train_df):,} samples ({len(train_df)/len(df):.0%})
Dev: {len(dev_df):,} samples ({len(dev_df)/len(df):.0%})
Test: {len(test_df):,} samples ({len(test_df)/len(df):.0%})
""")


Train: 1,131,158 samples (70%)
Dev: 242,391 samples (15%)
Test: 242,391 samples (15%)



##### **Encoding Using SBERT**

In [15]:
# SBERT model for Sentences Encoding
SBERT = SentenceTransformer('all-MiniLM-L6-v2')

def embed_text_batch(df, column_name, SBERT, batch_size=5000): # in batches for effeciency
    embeddings = []
    for i in tqdm(range(0, len(df), batch_size), desc=f"Encoding {column_name}"):
        batch = df[column_name].iloc[i:i+batch_size].tolist()
        emb = SBERT.encode(batch, show_progress_bar=False, device='cuda' if torch.cuda.is_available() else 'cpu')
        embeddings.append(emb)
        del batch, emb
        gc.collect()
    return np.vstack(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
def desc_embeddings(df, split_name):
    print(f"Creating SBERT Job Description embeddings for the {split_name}_df ...\n")
    desc_emb = embed_text_batch(df, 'cleaned_Job Description', SBERT)
    np.save(f'/kaggle/working/desc_emb_{split_name}.npy', desc_emb)
    del desc_emb
    gc.collect()
    
def resp_embeddings(df, split_name):
    print(f"Creating SBERT Responsibilities embeddings for the {split_name}_df ...\n")
    resp_emb = embed_text_batch(df, 'cleaned_Responsibilities', SBERT)
    np.save(f'/kaggle/working/resp_emb_{split_name}.npy', resp_emb)
    del resp_emb
    gc.collect()

In [17]:
splits = {
    'train': train_df,
    'dev': dev_df,
    'test': test_df
}

# Process and create embeddings for each split separately
for split_name, split_df in splits.items():
    temp_df = split_df.copy()
    desc_embeddings(temp_df, split_name)
    print("_"*100)
    resp_embeddings(temp_df, split_name)
    
    del temp_df
    gc.collect()

Creating SBERT Job Description embeddings for the train_df ...



Encoding cleaned_Job Description: 100%|██████████| 227/227 [07:57<00:00,  2.10s/it]


____________________________________________________________________________________________________
Creating SBERT Responsibilities embeddings for the train_df ...



Encoding cleaned_Responsibilities: 100%|██████████| 227/227 [07:55<00:00,  2.09s/it]


Creating SBERT Job Description embeddings for the dev_df ...



...

Encoding cleaned_Responsibilities: 100%|██████████| 49/49 [01:41<00:00,  2.08s/it]


Creating SBERT Job Description embeddings for the test_df ...



Encoding cleaned_Job Description: 100%|██████████| 49/49 [01:42<00:00,  2.09s/it]


____________________________________________________________________________________________________
Creating SBERT Responsibilities embeddings for the test_df ...



Encoding cleaned_Responsibilities: 100%|██████████| 49/49 [01:42<00:00,  2.09s/it]


##### **Dimensionality Reduction using SVD**

In [18]:
# loading embeddings
X_train_desc_emb = np.load("/kaggle/working/desc_emb_train.npy")
X_dev_desc_emb = np.load("/kaggle/working/desc_emb_dev.npy")
X_test_desc_emb = np.load("/kaggle/working/desc_emb_test.npy")

In [19]:
desc_svd = TruncatedSVD(n_components=200, random_state=42)
X_train_svd = desc_svd.fit_transform(X_train_desc_emb).astype(np.float16)
X_dev_svd = desc_svd.transform(X_dev_desc_emb).astype(np.float16)
X_test_svd = desc_svd.transform(X_test_desc_emb).astype(np.float16)

# Saving all splits
np.save('/kaggle/working/desc_emb_train.npy', X_train_svd)
np.save('/kaggle/working/desc_emb_dev.npy', X_dev_svd)
np.save('/kaggle/working/desc_emb_test.npy', X_test_svd)

# Saving the SVD model
joblib.dump(desc_svd, '/kaggle/working/desc_svd_model.joblib')

del X_train_svd, X_dev_svd, X_test_svd, desc_svd, X_train_desc_emb, X_dev_desc_emb, X_test_desc_emb
gc.collect()

33

In [20]:
# loading embeddings
X_train_resp_emb = np.load("/kaggle/working/resp_emb_train.npy")
X_dev_resp_emb = np.load("/kaggle/working/resp_emb_dev.npy")
X_test_resp_emb = np.load("/kaggle/working/resp_emb_test.npy")

In [21]:
resp_svd = TruncatedSVD(n_components=200, random_state=42)
X_train_svd = resp_svd.fit_transform(X_train_resp_emb).astype(np.float16)
X_dev_svd = resp_svd.transform(X_dev_resp_emb).astype(np.float16)
X_test_svd = resp_svd.transform(X_test_resp_emb).astype(np.float16)

# Saving all splits
np.save('/kaggle/working/resp_emb_train.npy', X_train_svd)
np.save('/kaggle/working/resp_emb_dev.npy', X_dev_svd)
np.save('/kaggle/working/resp_emb_test.npy', X_test_svd)

# Saving the SVD model
joblib.dump(resp_svd, '/kaggle/working/resp_svd_model.joblib')

del X_train_svd, X_dev_svd, X_test_svd, resp_svd, X_train_resp_emb, X_dev_resp_emb, X_test_resp_emb
gc.collect()

33

##### **SBERT Embeddings Fusion using PCA**

In [22]:
# 1. Loading all embeddings
desc_train = np.load('/kaggle/working/desc_emb_train.npy')
resp_train = np.load('/kaggle/working/resp_emb_train.npy')

desc_dev = np.load('/kaggle/working/desc_emb_dev.npy')
resp_dev = np.load('/kaggle/working/resp_emb_dev.npy')

desc_test = np.load('/kaggle/working/desc_emb_test.npy')
resp_test = np.load('/kaggle/working/resp_emb_test.npy')

# 2. Concatenate each split and clean
X_train_concat = np.hstack([desc_train, resp_train])
del desc_train, resp_train
gc.collect()

X_dev_concat = np.hstack([desc_dev, resp_dev])
del desc_dev, resp_dev
gc.collect()

X_test_concat = np.hstack([desc_test, resp_test])
del desc_test, resp_test
gc.collect()

0

In [23]:
# 3. Fitting PCA ONLY on train set
pca = PCA(n_components=200, random_state=42)
X_train_fused = pca.fit_transform(X_train_concat).astype(np.float16)
del X_train_concat
gc.collect()

# 4. Transform dev/test using the train-trained PCA
X_dev_fused = pca.transform(X_dev_concat).astype(np.float16)
del X_dev_concat
gc.collect()

X_test_fused = pca.transform(X_test_concat).astype(np.float16)
del X_test_concat
gc.collect()

0

##### **L2-Normalization for SBERT Embeddings**

In [24]:
# 5. L2-normalize all splits
X_train_fused = normalize(X_train_fused, norm='l2')
X_dev_fused = normalize(X_dev_fused, norm='l2')
X_test_fused = normalize(X_test_fused, norm='l2')

# 6. Save results
np.save('/kaggle/working/fused_emb_train.npy', X_train_fused)
np.save('/kaggle/working/fused_emb_dev.npy', X_dev_fused)
np.save('/kaggle/working/fused_emb_test.npy', X_test_fused)

del X_train_fused, X_dev_fused, X_test_fused
gc.collect()

# 7. Saving PCA model for future inference
joblib.dump(pca, '/kaggle/working/pca_fusion_model.joblib')

['/kaggle/working/pca_fusion_model.joblib']

In [25]:
splits = {
    'train': train_df,
    'dev': dev_df,
    'test': test_df
}

# Process and save each split separately
for split_name, split_df in splits.items():
    split_df.drop(columns=['Job Description', 'Responsibilities'], inplace=True)
    split_df.to_csv(f'/kaggle/working/{split_name}_data.csv', index=False)

#### 2) Columns: ***Skills & Benefits***  

In [26]:
# 1. Skills -> TF-IDF (L2 normalized by default)

# First clean all text
for split_df in [train_df, dev_df, test_df]:
    split_df['skills_clean'] = split_df['cleaned_skills'].str.lower().str.replace(r'[^\w\s,]', '', regex=True)

# Fitting TF-IDF ONLY on train
tfidf = TfidfVectorizer(
    max_features=300,
    stop_words='english',
    ngram_range=(1, 2),
    token_pattern=r'(?u)\b\w+\b'
)
train_skills = tfidf.fit_transform(train_df['skills_clean'])

# Transform other splits
dev_skills = tfidf.transform(dev_df['skills_clean'])
test_skills = tfidf.transform(test_df['skills_clean'])

# Saving each split separately
scipy.sparse.save_npz('/kaggle/working/skills_tfidf_train.npz', train_skills)
scipy.sparse.save_npz('/kaggle/working/skills_tfidf_dev.npz', dev_skills)
scipy.sparse.save_npz('/kaggle/working/skills_tfidf_test.npz', test_skills)

# Saving the TF-IDF vectorizer
joblib.dump(tfidf, '/kaggle/working/tfidf_vectorizer.joblib')

# Cleanup
for split_df in [train_df, dev_df, test_df]:
    split_df.drop(columns=['skills', 'skills_clean'], inplace=True)

del train_skills, dev_skills, test_skills, tfidf
gc.collect()

391

In [27]:
# 2. BENEFITS → Multi-hot
def extract_benefits(text):
    # Removing curly braces and quotes, then splitting (they are in JSON format)
    if pd.isna(text): return []
    return [b.strip().lower() for b in re.sub(r"[{}']", "", text).split(',') if b.strip()]

# Fitting MultiLabelBinarizer ONLY on train
mlb = MultiLabelBinarizer()
train_benefits = mlb.fit_transform(train_df['Benefits'].apply(extract_benefits))

In [28]:
# Create column names
benefits_cols = [f"benefit_{b}" for b in mlb.classes_]

# Add to each DataFrame
for split_df, benefits in zip(
    [train_df, dev_df, test_df],
    [
        train_benefits,
        mlb.transform(dev_df['Benefits'].apply(extract_benefits)),
        mlb.transform(test_df['Benefits'].apply(extract_benefits))
    ]
):
    # Add benefits columns
    split_df[benefits_cols] = pd.DataFrame(benefits, columns=benefits_cols, index=split_df.index)
    
    # Drop original column
    split_df.drop(columns=['Benefits'], inplace=True)

# Checkpoint, Save the updated DataFrames
train_df.to_csv('/kaggle/working/train_data.csv', index=False)
dev_df.to_csv('/kaggle/working/dev_data.csv', index=False)
test_df.to_csv('/kaggle/working/test_data.csv', index=False)

# Save the binarizer
joblib.dump(mlb, '/kaggle/working/benefits_multilabel_binarizer.joblib')

del mlb, train_benefits
gc.collect()

0

In [29]:
splits = {
    'train': train_df,
    'dev': dev_df,
    'test': test_df
}
for split_name, split_df in splits.items():
    split_df.drop(columns=['cleaned_Job Description', 'cleaned_Responsibilities', 'cleaned_skills'], inplace=True)
    split_df.to_csv(f'/kaggle/working/{split_name}_data.csv', index=False)

## **3rd | Smoothing Outliers & Normalization**

In [30]:
def normalize_numeric(train_df, dev_df=None, test_df=None, mode='fit'):
    """
    - Winsorization (all numeric cols)
    - QuantileTransformer (frequency cols)
    - RobustScaler (other numeric cols)
    """
    # columns
    freq_cols = ['Role_freq', 'Company_freq']
    benefit_cols = [col for col in train_df.columns if col.startswith('benefit_')]
    exclude_cols = ['Job Title_encoded', 'salary_avg'] + benefit_cols
    numeric_cols = [col for col in train_df.columns 
                   if col not in exclude_cols 
                   and pd.api.types.is_numeric_dtype(train_df[col])]
    
    # Separate frequency columns
    standard_cols = [col for col in numeric_cols if col not in freq_cols]
    
    # 1. Winsorization (all splits, all numeric cols)
    def winsorize(s, lower=0.01, upper=0.99):
        q = s.quantile([lower, upper])
        return s.clip(q.iloc[0], q.iloc[1])
    
    for df_split in [d for d in [train_df, dev_df, test_df] if d is not None]:
        for col in numeric_cols:
            df_split[col] = winsorize(df_split[col])
    
    # 2. Transformers
    transformers = [
        ('freq', QuantileTransformer(n_quantiles=100, output_distribution='uniform'), freq_cols),
        ('robust', RobustScaler(), standard_cols)
    ]
    preprocessor = ColumnTransformer(transformers, remainder='passthrough')
    
    # 3. Fit/transform
    if mode in ('fit', 'all'):
        train_df[numeric_cols] = preprocessor.fit_transform(train_df[numeric_cols])
        joblib.dump(preprocessor, '/kaggle/working/numeric_preprocessor.joblib')
    else:
        preprocessor = joblib.load('/kaggle/working/numeric_preprocessor.joblib')
    
    if dev_df is not None:
        dev_df[numeric_cols] = preprocessor.transform(dev_df[numeric_cols])
    if test_df is not None:
        test_df[numeric_cols] = preprocessor.transform(test_df[numeric_cols])
    
    print(f"Processed cols: {len(freq_cols)} freq (Quantile) + {len(standard_cols)} standard (Robust)")

# Usage
normalize_numeric(train_df, dev_df, test_df, mode='fit')

train_df.to_csv('/kaggle/working/train_data.csv', index=False)
dev_df.to_csv('/kaggle/working/dev_data.csv', index=False)
test_df.to_csv('/kaggle/working/test_data.csv', index=False)

train_df.to_parquet('/kaggle/working/train_data.parquet', index=False)
dev_df.to_parquet('/kaggle/working/dev_data.parquet', index=False)
test_df.to_parquet('/kaggle/working/test_data.parquet', index=False)

Processed cols: 2 freq (Quantile) + 12 standard (Robust)


In [31]:
train_df.head(1)

,Qualifications,Country,Work Type,Company Size,Preference,Job Portal,exp_min,exp_max,exp_avg,salary_min,...,benefit_parental leave,benefit_professional development,benefit_profit-sharing,benefit_relocation assistance,benefit_retirement plans,benefit_social and recreational activities,benefit_stock options or equity grants,benefit_transportation benefits,benefit_tuition reimbursement,benefit_wellness programs
473765,0.577778,0.30303,0.0,0.453704,-0.5,0.968005,0.5,0.625,-0.666667,0.0,...,0,1,0,1,0,0,0,0,0,0


In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1131158 entries, 473765 to 825460
Data columns (total 41 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   Qualifications                              1131158 non-null  float64
 1   Country                                     1131158 non-null  float64
 2   Work Type                                   1131158 non-null  float64
 3   Company Size                                1131158 non-null  float64
 4   Preference                                  1131158 non-null  float64
 5   Job Portal                                  1131158 non-null  float64
 6   exp_min                                     1131158 non-null  float64
 7   exp_max                                     1131158 non-null  float64
 8   exp_avg                                     1131158 non-null  float64
 9   salary_min                                  1131158 non-nu